# Gestión de Alumnos - Programación 4

Este notebook contiene herramientas para gestionar información de alumnos, incluyendo:

- **Lectura**: Parsing de archivos Markdown con datos de estudiantes
- **Escritura**: Generación de archivos formateados
- **Organización**: Creación automática de estructura de carpetas
- **Normalización**: Estandarización de formato de archivos

## Funciones principales

1. `leer()` - Lee y parsea archivos Markdown
2. `escribir()` - Genera archivos formateados  
3. `normalizar()` - Estandariza formato de archivos
4. `crear_carpetas()` - Organiza estructura de directorios

In [ ]:
import re
import os
from pathlib import Path
from itertools import zip_longest
from typing import List, Dict


def leer(origen: str = "alumnos.md") -> list:
    salida, comision = [], ""
    with open(origen, "r", encoding='utf-8') as archivo:
        for linea in archivo:
            if not linea.strip():
                continue

            if linea.startswith("##"):
                comision = linea.strip("# \n")

            if linea.startswith("- "):
                valores = re.split(r"\s{2,}", linea.strip("- \n")) + [comision]
                valores = dict(zip('legajo nombre telefono comision'.split(), valores))
                salida.append(valores)

    return sorted(salida, key=lambda x: [x['comision'], x['legajo']])


def escribir(alumnos, destino="alumnos2.md"):
    with open(destino, "w", encoding='utf-8') as archivo:
        archivo.write("# Alumnos Programación 4\n")
        for comision in sorted(set(alumno['comision'] for alumno in alumnos)):
            archivo.write(f"\n## {comision}\n")
            archivo.write("```\n")
            for alumno in alumnos:
                if alumno['comision'] == comision:
                    archivo.write(f"- {alumno['legajo']}  {alumno['nombre']:40}  {alumno['telefono']}\n")
            archivo.write("```\n")
    return alumnos


def normalizar(origen='alumnos.md'):
    return escribir(leer(origen), origen)

def crear_carpetas(alumnos, base = 'tp'):
    for a in alumnos:
        nuevo = Path(base) / f"{a['legajo']} - {a['nombre']}"
        for viejo in Path(base).glob(f"{a['legajo']} -*"):
            if viejo != nuevo: viejo.rename(nuevo)
        else:
            nuevo.mkdir(parents=True, exist_ok=True)


crear_carpetas(leer())

## Ejemplo de uso

El código anterior procesa automáticamente el archivo `alumnos.md` y crea las carpetas correspondientes para cada estudiante en el directorio `tp/`.

Para usar las funciones individualmente:

```python
# Leer datos de alumnos
alumnos = leer('alumnos.md')

# Crear estructura de carpetas
crear_carpetas(alumnos, 'tp')

# Normalizar archivo existente
normalizar('alumnos.md')
```